In [1]:
!pip install colorama

In [2]:
import pandas as pd
import numpy as np


from matplotlib.ticker import MaxNLocator
import seaborn as sns
from cycler import cycler
from IPython.display import display
import math
import os
import random
import gc
import sys
import warnings
warnings.filterwarnings('ignore')

import optuna
from colorama import Fore, Back, Style

from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold, train_test_split,GroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.calibration import CalibrationDisplay
from sklearn.preprocessing import StandardScaler,RobustScaler,LabelEncoder
from sklearn.impute import KNNImputer
from sklearn import linear_model
from sklearn.linear_model import HuberRegressor
from sklearn.decomposition import PCA
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.calibration import CalibratedClassifierCV

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Activation,  BatchNormalization, Dropout, Concatenate, Embedding,  Flatten, Conv1D
from tensorflow.keras.models import Model

In [7]:
train = pd.read_csv(r'C:\Users\satya\Downloads\tabular-playground-series-aug-2022\train.csv')
test = pd.read_csv(r'C:\Users\satya\Downloads\tabular-playground-series-aug-2022\test.csv')
submission = pd.read_csv(r'C:\Users\satya\Downloads\tabular-playground-series-aug-2022\sample_submission.csv')
target = train['failure']
train.drop('failure',axis=1, inplace = True)
data = pd.concat([train, test])
train.shape,test.shape

((26570, 25), (20775, 25))

<h1> Preprocessing


Initial preprocessing from :

https://www.kaggle.com/code/desalegngeb/tps08-logisticregression-and-some-fe

https://www.kaggle.com/code/alikayed/tps08-logisticregression-and-some-fe-c83a47

In [8]:
# library for coding string values :
! pip install feature_engine
from feature_engine.encoding import WoEEncoder

In [9]:
data['m3_missing'] = data['measurement_3'].isnull().astype(np.int8)
data['m5_missing'] = data['measurement_5'].isnull().astype(np.int8)
data['area'] = data['attribute_2'] * data['attribute_3']

feature = [f for f in test.columns if f.startswith('measurement') or f=='loading']

# dictionnary of dictionnaries (for the 11 best correlated measurement columns), 
# we will use the dictionnaries below to select the best correlated columns according to the product code)
# Only for 'measurement_17' we make a 'manual' selection :
full_fill_dict ={}
full_fill_dict['measurement_17'] = {
    'A': ['measurement_5','measurement_6','measurement_8'],
    'B': ['measurement_4','measurement_5','measurement_7'],
    'C': ['measurement_5','measurement_7','measurement_8','measurement_9'],
    'D': ['measurement_5','measurement_6','measurement_7','measurement_8'],
    'E': ['measurement_4','measurement_5','measurement_6','measurement_8'],
    'F': ['measurement_4','measurement_5','measurement_6','measurement_7'],
    'G': ['measurement_4','measurement_6','measurement_8','measurement_9'],
    'H': ['measurement_4','measurement_5','measurement_7','measurement_8','measurement_9'],
    'I': ['measurement_3','measurement_7','measurement_8']
}

# collect the name of the next 10 best measurement columns sorted by correlation (except 17 already done above):
col = [col for col in test.columns if 'measurement' not in col]+ ['loading','m3_missing','m5_missing']
a = []
b =[]
for x in range(3,17):
    corr = np.absolute(data.drop(col, axis=1).corr()[f'measurement_{x}']).sort_values(ascending=False)
    a.append(np.round(np.sum(corr[1:4]),3)) # we add the 3 first lines of the correlation values to get the "most correlated"
    b.append(f'measurement_{x}')
c = pd.DataFrame()
c['Selected columns'] = b
c['correlation total'] = a
c = c.sort_values(by = 'correlation total',ascending=False).reset_index(drop = True)
print(f'Columns selected by correlation sum of the 3 first rows : ')
display(c.head(10))

for i in range(10):
    measurement_col = 'measurement_' + c.iloc[i,0][12:] # we select the next best correlated column 
    fill_dict ={}
    for x in data.product_code.unique() : 
        corr = np.absolute(data[data.product_code == x].drop(col, axis=1).corr()[measurement_col]).sort_values(ascending=False)
        measurement_col_dic = {}
        measurement_col_dic[measurement_col] = corr[1:5].index.tolist()
        fill_dict[x] = measurement_col_dic[measurement_col]
    full_fill_dict[measurement_col] =fill_dict
    
feature = [f for f in data.columns if f.startswith('measurement') or f=='loading']
nullValue_cols = [col for col in train.columns if train[col].isnull().sum()!=0]
    
for code in data.product_code.unique():
    total_na_filled_by_linear_model = 0
    print(f'\n-------- Product code {code} ----------\n')
    print(f'filled by linear model :')
    for measurement_col in list(full_fill_dict.keys()):
        tmp = data[data.product_code==code]
        column = full_fill_dict[measurement_col][code]
        tmp_train = tmp[column+[measurement_col]].dropna(how='any')
        tmp_test = tmp[(tmp[column].isnull().sum(axis=1)==0)&(tmp[measurement_col].isnull())]

        model = HuberRegressor(epsilon=1.9)
        model.fit(tmp_train[column], tmp_train[measurement_col])
        data.loc[(data.product_code==code)&(data[column].isnull().sum(axis=1)==0)&(data[measurement_col].isnull()),measurement_col] = model.predict(tmp_test[column])
        print(f'{measurement_col} : {len(tmp_test)}')
        total_na_filled_by_linear_model += len(tmp_test)
        
    # others NA columns:
    NA = data.loc[data["product_code"] == code,nullValue_cols ].isnull().sum().sum()
    model1 = KNNImputer(n_neighbors=3)
    data.loc[data.product_code==code, feature] = model1.fit_transform(data.loc[data.product_code==code, feature])
    print(f'\n{total_na_filled_by_linear_model} filled by linear model ') 
    print(f'{NA} filled by KNN ')
    
data['measurement_avg'] = data[[f'measurement_{i}' for i in range(3, 17)]].mean(axis=1)

Columns selected by correlation sum of the 3 first rows : 


,Selected columns,correlation total
0,measurement_8,0.448
1,measurement_11,0.395
2,measurement_5,0.376
3,measurement_6,0.359
4,measurement_7,0.330
5,measurement_4,0.328
6,measurement_15,0.301
7,measurement_10,0.300
8,measurement_16,0.252
9,measurement_14,0.225



-------- Product code A ----------

filled by linear model :
measurement_17 : 386
measurement_8 : 167
measurement_11 : 225
measurement_5 : 113
measurement_6 : 146
measurement_7 : 153
measurement_4 : 79
measurement_15 : 273
measurement_10 : 209
measurement_16 : 293
measurement_14 : 237

2281 filled by linear model 
1568 filled by KNN 

-------- Product code B ----------

filled by linear model :
measurement_17 : 418
measurement_8 : 165
measurement_11 : 220
measurement_5 : 83
measurement_6 : 106
measurement_7 : 174
measurement_4 : 80
measurement_15 : 294
measurement_10 : 197
measurement_16 : 358
measurement_14 : 330

2425 filled by linear model 
1550 filled by KNN 

-------- Product code C ----------

filled by linear model :
measurement_17 : 391
measurement_8 : 189
measurement_11 : 231
measurement_5 : 141
measurement_6 : 150
measurement_7 : 140
measurement_4 : 108
measurement_15 : 319
measurement_10 : 262
measurement_16 : 343
measurement_14 : 330

2604 filled by linear model 
1740 fill

In [ ]:
def _scale(train_data, val_data, test_data, feats):
    scaler = StandardScaler()
    # scaler = PowerTransformer()
    
    scaled_train = scaler.fit_transform(train_data[feats])
    scaled_val = scaler.transform(val_data[feats])
    scaled_test = scaler.transform(test_data[feats])
    
    #back to dataframe
    new_train = train_data.copy()
    new_val = val_data.copy()
    new_test = test_data.copy()
    
    new_train[feats] = scaled_train
    new_val[feats] = scaled_val
    new_test[feats] = scaled_test
    
    assert len(train_data) == len(new_train)
    assert len(val_data) == len(new_val)
    assert len(test_data) == len(new_test)
    
    return new_train, new_val, new_test

In [ ]:
train = data.iloc[:train.shape[0],:]
test = data.iloc[train.shape[0]:,:]
print(train.shape, test.shape)

groups = train.product_code
X = train
y = target

In [ ]:
#Thanks to @MAXSARMENTO 
woe_encoder = WoEEncoder(variables=['attribute_0'])
woe_encoder.fit(X, y)
X = woe_encoder.transform(X)
test = woe_encoder.transform(test)

In [ ]:
select_feature = ['loading',
                  'attribute_0',
                  'measurement_17',
                  'measurement_0',
                  'measurement_1',
                  'measurement_2',
                  'area',
                  'm3_missing',
                  'm5_missing',
                  'measurement_avg']

In [ ]:
seeds = [4, 8, 12, 24, 48, 64, 86, 128, 256]
lr_test = np.zeros(len(test))
lr_auc = 0

for i, seed in enumerate(seeds):
    print('------- Seed:%2d -------' % (seeds[i]))
    np.random.seed(seed)

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X, y,groups=train.product_code)):
        x_train, x_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        x_train, x_val, x_test = _scale(x_train, x_val, test, select_feature)

        model = linear_model.LogisticRegression(max_iter=200, C=0.0001, penalty='l2', solver='newton-cg')
        model.fit(x_train[select_feature], y_train)

        val_preds = model.predict_proba(x_val[select_feature])[:, 1]
        roc = round(roc_auc_score(y_val, val_preds), 5)
        print("FOLD: ", fold_idx+1, " ROC-AUC:", roc)
        lr_auc += roc_auc_score(y_val, val_preds) / (5 * 5)
        lr_test += model.predict_proba(x_test[select_feature])[:, 1] / len(seeds)
        
    print('')

In [ ]:
submission['failure'] = lr_test
submission.to_csv(f"./submission.csv", index=False)